In [1]:
import numpy as np
import pandas as pd
import os
from google.colab import drive

In [2]:
!pip install iterative-stratification
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [3]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/DIPROMATS_2024/data/')

Mounted at /content/drive


Cargar el dataset de train:

In [4]:
df_train_en = pd.read_json("dipromats24_t1_train_en.json")
df_train_es = pd.read_json("dipromats24_t1_train_es.json")
frames = [df_train_en, df_train_es]
dataset = pd.concat(frames)
dataset = dataset.sample(frac = 1).reset_index(drop=True)
print('dataset size:', dataset.shape)
dataset.head()

dataset size: (14528, 13)


,test_case,id,country,username,tweet_type,tweet_id,UTC,rts&fav,language,text,label_task1,label_task2,label_task3
0,DIPROMATS2024-TASK1,4374,European Union,eu_commission,Tweet,1274312850786250752,2020-06-20 12:07:13+00:00,183,en,This week EU countries took the first step tow...,False,[],[]
1,DIPROMATS2024-TASK1,986,USA,secpompeo,Tweet,1228656724061675520,2020-02-15 12:26:05+00:00,2805,en,The United States is fighting for sovereignty ...,True,[1 appeal to commonality],[1 appeal to commonality - flag waving]
2,DIPROMATS2024-TASK1,5615,USA,usambnmacedonia,Tweet,1288848693983170560,2020-07-30 14:47:28+00:00,136,en,Returned to #Bitola last weekend to take in th...,False,[],[]
3,DIPROMATS2024-TASK1,5672,Russia,russia,Tweet,1289806116545732608,2020-08-02 06:11:56+00:00,227,en,"👨‍🚀 Soviet cosmonaut №5⃣, legendary Valery #By...",True,"[1 appeal to commonality, 3 loaded language]","[1 appeal to commonality - flag waving, 3 load..."
4,DIPROMATS2024-TASK1,7075,Russia,embassyofrussia,Tweet,1306891024225251328,2020-09-18 09:41:15+00:00,42,en,"#OTD in 1918, 102 years ago, Lieutenant Viktor...",True,[1 appeal to commonality],[1 appeal to commonality - flag waving]


In [5]:
count = 0
for i in range(len(dataset['label_task2'])):
  if dataset['label_task1'][i] == False:
    dataset['label_task2'][i] = '0 not propagandistic'
  else:
    label_str = dataset['label_task2'][i][0]
    for j in range(1, len(dataset['label_task2'][i])):
      label_str = label_str + ',' + dataset['label_task2'][i][j]
    dataset['label_task2'][i] = label_str

Se han truncado las últimas 5000 líneas del flujo de salida.
<ipython-input-5-29c6a175c2eb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['label_task2'][i] = '0 not propagandistic'
<ipython-input-5-29c6a175c2eb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['label_task2'][i] = '0 not propagandistic'
<ipython-input-5-29c6a175c2eb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['label_task2'][i] = '0 not p

In [6]:
import collections
labels = dataset['label_task2'].to_list()
c = collections.Counter(labels)
c

Counter({'0 not propagandistic': 11361,
         '1 appeal to commonality': 556,
         '1 appeal to commonality,3 loaded language': 232,
         '2 discrediting the opponent': 1240,
         '3 loaded language': 798,
         '1 appeal to commonality,2 discrediting the opponent,3 loaded language': 41,
         '2 discrediting the opponent,3 loaded language': 231,
         '1 appeal to commonality,2 discrediting the opponent': 69})

In [7]:
sep_labels = []
for label in labels:
    sep_labels.extend(label.split(','))
cs = collections.Counter(sep_labels)
cs

Counter({'0 not propagandistic': 11361,
         '1 appeal to commonality': 898,
         '3 loaded language': 1302,
         '2 discrediting the opponent': 1581})

In [8]:
key = {label:i for i,label in enumerate(cs.keys())}
key

{'0 not propagandistic': 0,
 '1 appeal to commonality': 1,
 '3 loaded language': 2,
 '2 discrediting the opponent': 3}

In [9]:
text_to_category = {label:[] for label in cs.keys()}
for idx, item in dataset.iterrows():
    for label in text_to_category:
        if label in item['label_task2']:
            text_to_category[label].append(1)
        else:
            text_to_category[label].append(0)

for label in text_to_category:
    dataset[label] = text_to_category[label]

División train-validation-test para la tarea 1a (70-15-15)

In [10]:
from sklearn.model_selection import train_test_split
X=dataset.drop('label_task1', axis=1)
y=dataset['label_task1']

X_t1_train, X_aux, y_t1_train, y_aux = train_test_split(X, y,
        test_size=0.3, random_state=42, stratify=y)
X_t1_val, X_t1_test, y_t1_val, y_t1_test = train_test_split(X_aux, y_aux,
        test_size=1/2, random_state=42, stratify=y_aux)

train_t1_df = pd.DataFrame({'label_task1': y_t1_train})
train_t1_df = pd.concat([X_t1_train, train_t1_df], axis=1)
train_t1_df = train_t1_df[['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3']]

val_t1_df = pd.DataFrame({'label_task1': y_t1_val})
val_t1_df = pd.concat([X_t1_val, val_t1_df], axis=1)
val_t1_df = val_t1_df[['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3']]

test_t1_df = pd.DataFrame({'label_task1': y_t1_test})
test_t1_df = pd.concat([X_t1_test, test_t1_df], axis=1)
test_t1_df = test_t1_df[['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3']]

División train-validation-test para la tarea 1b (75-12-13)

In [11]:
X_t2,Y_t2 = dataset.drop(["0 not propagandistic", "1 appeal to commonality", "2 discrediting the opponent", "3 loaded language"], axis=1).to_numpy(), dataset[["0 not propagandistic", "1 appeal to commonality", "2 discrediting the opponent", "3 loaded language"]].to_numpy(dtype=np.float32)

In [12]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

msss = MultilabelStratifiedKFold(n_splits=4, shuffle=True, random_state=42)

for train_index, test_index in msss.split(X_t2, Y_t2):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_t2_train, X_t2_test = X_t2[train_index], X_t2[test_index]
    y_t2_train, y_t2_test = Y_t2[train_index], Y_t2[test_index]

    train_t2_df = pd.DataFrame(columns=["0 not propagandistic", "1 appeal to commonality", "2 discrediting the opponent", "3 loaded language"], data=y_t2_train)
    aux_df = pd.DataFrame(columns=['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', 'label_task3'], data=X_t2_train)
    train_t2_df = pd.concat([aux_df, train_t2_df], axis=1)
    train_t2_df = train_t2_df[['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3']]

    break

TRAIN: [    0     1     3 ... 14522 14526 14527] TEST: [    2     8     9 ... 14523 14524 14525]


In [13]:
msss_2 = MultilabelStratifiedKFold(n_splits=2, shuffle=True, random_state=42)

for val_index, test_index in msss_2.split(X_t2_test, y_t2_test):
    print("VALIDATION:", val_index, "TEST:", test_index)
    X_t2_val, X_t2_test_2 = X_t2_test[val_index], X_t2_test[test_index]
    y_t2_val, y_t2_test_2 = y_t2_test[val_index], y_t2_test[test_index]

    val_t2_df = pd.DataFrame(columns=["0 not propagandistic", "1 appeal to commonality", "2 discrediting the opponent", "3 loaded language"], data=y_t2_val)
    aux_df = pd.DataFrame(columns=['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', 'label_task3'], data=X_t2_val)
    val_t2_df = pd.concat([aux_df, val_t2_df], axis=1)
    val_t2_df = val_t2_df[['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3']]

    test_t2_df = pd.DataFrame(columns=["0 not propagandistic", "1 appeal to commonality", "2 discrediting the opponent", "3 loaded language"], data=y_t2_test)
    aux_df = pd.DataFrame(columns=['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', 'label_task3'], data=X_t2_test)
    test_t2_df = pd.concat([aux_df, test_t2_df], axis=1)
    test_t2_df = test_t2_df[['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3']]

    break

VALIDATION: [   1    2    3 ... 3626 3630 3632] TEST: [   0    7   12 ... 3628 3629 3631]


Cargar el dataset de test:

In [14]:
df_test_en = pd.read_json("dipromats24_t1_test_en.json")
df_test_es = pd.read_json("dipromats24_t1_test_es.json")
frames = [df_test_en, df_test_es]
dataset_test = pd.concat(frames)
dataset_test = dataset_test.sample(frac = 1).reset_index(drop=True)
print('dataset test size:', dataset_test.shape)
dataset_test.head()

dataset test size: (7075, 10)


,test_case,id,country,user_name,tweet_type,tweet_id,UTC,rts&fav,language,text
0,DIPROMATS2024-TASK1,8982,USA,usembassyguate,Tweet,1359998157673889793,2021-02-11 22:50:02+00:00,226,es,El gobierno de Biden ha reinstaurado el compro...
1,DIPROMATS2024-TASK1,6975,China,chinaembarg,Retweet,1320443364035223554,2020-10-25 19:13:25+00:00,0,es,Presentamos oficialmente el stand que tendrá M...
2,DIPROMATS2024-TASK1,8733,European Union,ueenmexico,quoted,1356278892194127872,2021-02-01 16:31:00+00:00,11,es,"""Condeno enérgicamente el golpe de Estado llev..."
3,DIPROMATS2024-TASK1,9104,China,spokespersonchn,Tweet,1333041772696571904,2020-11-29 13:34:59+00:00,1233,en,According to the #SouthChinaSea Strategic Situ...
4,DIPROMATS2024-TASK1,8604,European Union,ueenmexico,Tweet,1353384410012397569,2021-01-24 16:49:22+00:00,17,es,¡Feliz Día Internacional de la Educación! 🙌📚\n...


Subir el dataset a HuggingFace:

In [27]:
from huggingface_hub import login
login()

In [28]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset, load_from_disk

t1_train = train_t1_df.to_dict()
t1_val = val_t1_df.to_dict()
t1_test = test_t1_df.to_dict()
t2_train = train_t2_df.to_dict()
t2_val = val_t2_df.to_dict()
t2_test = test_t2_df.to_dict()

task1 = { 'train':Dataset.from_pandas(train_t1_df),
          'val':Dataset.from_pandas(val_t1_df),
          'test':Dataset.from_pandas(test_t1_df)
        }
task2 = { 'train':Dataset.from_pandas(train_t2_df),
          'val':Dataset.from_pandas(val_t2_df),
          'test':Dataset.from_pandas(test_t2_df)
        }

task1 = DatasetDict(task1)
task2 = DatasetDict(task2)
task1.push_to_hub("UC3M-LCPM/DIPROMATS_2024", "task1a", private=True)
task2.push_to_hub("UC3M-LCPM/DIPROMATS_2024", "task1b", private=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/6.39k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/UC3M-LCPM/DIPROMATS_2024/commit/787a30606e0e42b759ae6647264cf14a58b87a0a', commit_message='Upload dataset', commit_description='', oid='787a30606e0e42b759ae6647264cf14a58b87a0a', pr_url=None, pr_revision=None, pr_num=None)

Comprobación de que el dataset se ha subido bien a HuggingFace y que se puede cargar en el notebook:

In [ ]:
from datasets import load_dataset

dataset = load_dataset("UC3M-LCPM/DIPROMATS_2024", 'task1a')

Generating train split:   0%|          | 0/10169 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/2179 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2180 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3'],
        num_rows: 10169
    })
    val: Dataset({
        features: ['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3'],
        num_rows: 2179
    })
    test: Dataset({
        features: ['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label_task1', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3'],
        num_rows: 2180
    })
})

Hasta aquí para subir a HuggingFace el modelo.

Distribución de las etiquetas para la tarea 1b:

In [ ]:
import plotly.express as px

In [ ]:
fig = px.parallel_categories(dataset[["0 not propagandistic", "1 appeal to commonality", "2 discrediting the opponent", "3 loaded language"]], color="3 loaded language", color_continuous_scale="sunset",\
                             title="Distribution of labels task 2")
fig.show()

Para guardarlo como JSON en local:

In [ ]:
import json

t1_train = train_t1_df.to_json()
t1_val = val_t1_df.to_json()
t1_test = test_t1_df.to_json()
t2_train = train_t2_df.to_json()
t2_val = val_t2_df.to_json()
t2_test = test_t2_df.to_json()

task1 ={'train':t1_train,
        'val':t1_val,
        'test':t1_test
        }
task2 = {'train':t2_train,
         'val':t2_val,
         'test':t2_test
        }
task1 = json.dumps(task1)
task1 = json.loads(task1)
task2 = json.dumps(task2)
task2 = json.loads(task2)

d = {'task1': task1,
     'task2': task2}

d = json.dumps(d)
d = json.loads(d)

In [ ]:
with open('dipromats24_t1_train.json', 'w') as f:
    json.dump(task1, f)

with open('dipromats24_t2_train.json', 'w') as g:
    json.dump(task2, g)